In [17]:
import argparse
import h5py
import numpy as np
import os
import pickle
import shutil
import yaml
import json
import click
import re
import random
from IPython.display import Image, display
from pprint import pprint
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch import Tensor
import vqa.lib.engine as engine
import vqa.lib.utils as utils
import vqa.lib.logger as logger
import vqa.lib.criterions as criterions
import vqa.datasets as datasets
import vqa.models as models

In [2]:
with open('options/cx/counterexamples_default.yaml', 'r') as handle:
    options = yaml.load(handle)
options['vgenome'] = None

# Load valset

In [6]:
val_examples_list = pickle.load(open('data/vqa2/processed/nans,2000_maxlength,26_minwcount,0_nlp,mcb_pad,right_trainsplit,train/valset.pickle', 'rb'))
qid_to_example = {ex['question_id']: ex for ex in val_examples_list}

In [7]:
# http://visualqa.org/data/mscoco/vqa/v2_Complementary_Pairs_val_mscoco.zip
comp_pairs = json.load(open("data/vqa2/raw/annotations/v2_mscoco_val2014_complementary_pairs.json", "r"))

comp_q = {}
for q1, q2 in comp_pairs:
    comp_q[q1] = q2
    comp_q[q2] = q1
    
len(comp_q)

188030

# Load KNNs data

In [8]:
# http://visualqa.org/data/mscoco/vqa/mscoco_val2014_nn_images.json
"""
The data is in dictionary format where each key is a COCO image id and each value is a list of 24 COCO image ids 
which are 24 nearest neighbor images of the key image id sorted in increasing distance, i.e., the first image in 
the list is the 1st nearest neighbor image and the last image in the list is 24th nearest neighbor image.
"""

knns = json.load(open("data/coco/knn/mscoco_val2014_nn_images.json", "r"))
knns = {int(k):v for k,v in knns.items()}

In [9]:
def coco_name_to_num(name):
    assert(name[-4:] == '.jpg')
    assert(name[-17] == '_')
    return int(name[-16:-4])

def coco_num_to_name(num, split='val'):
    if len(str(num)) > 12:
        raise ValueError
    if split == 'train':
        return 'COCO_train2014_{}.jpg'.format(str(num).zfill(12))
    elif split == 'val':
        return 'COCO_val2014_{}.jpg'.format(str(num).zfill(12))
    else:
        raise ValueError('split must be train or val; got {}'.format(split))
    
print(coco_name_to_num('COCO_val2014_000000452014.jpg'))
print(coco_num_to_name(452014))
print(coco_num_to_name(452014, 'val'))

452014
COCO_val2014_000000452014.jpg
COCO_val2014_000000452014.jpg


# Filter down examples

In [10]:
# Get all unique q_ids in the complimentary pairs data
q_ids_alpha = []
for q1, q2 in comp_pairs:
    q_ids_alpha.append(q1)
    q_ids_alpha.append(q2)
q_ids_alpha_set = set(q_ids_alpha)

In [11]:
# Get all unique q_ids in the VQA2 valing data
q_id_to_example = {ex['question_id']: ex for ex in val_examples_list}
q_ids_beta_set = set(q_id_to_example.keys())

In [13]:
# Approx 1.23% of the val example q_ids are not present in the complimentary pairs data
print(1 - len(q_ids_alpha_set) / len(q_ids_beta_set))

0.12280619909122292


In [14]:
# Additionally, the alpha and beta sets are only partially overlapping
intersection = q_ids_alpha_set.intersection(q_ids_beta_set)
print(len(intersection), len(q_ids_beta_set - q_ids_alpha_set), len(q_ids_alpha_set - q_ids_beta_set))

188030 26324 0


In [15]:
val_examples_filtered = []

for ex in val_examples_list:
    
    q1 = ex['question_id']
    i1 = coco_name_to_num(ex['image_name'])
    
    # Ensure q1 has a compliment
    if q1 in intersection:
        q2 = comp_q[q1]
        
        # Ensure q2 has a compliment and is in the valing data
        if q2 in intersection:
            
            ex2 = qid_to_example[q2]
            i2 = coco_name_to_num(ex2['image_name'])
            
            # Ensure i2 is in i1's KNNs
            if i2 in knns[i1]:
                
                ex['comp'] = ex2
                ex['comp']['knn_index'] = knns[i1].index(i2)
                ex['knns'] = [coco_num_to_name(i) for i in knns[i1]]
                
                val_examples_filtered.append(ex)
            
len(val_examples_filtered)

118499

In [16]:
# Sanity check: ensure all images in the filtered valset are accounted for in the KNNs data

# All unique image ids in filtered valset
v_nums_charlie = set([coco_name_to_num(ex['image_name']) for ex in val_examples_filtered])

# All unique image ids in KNNs data
v_nums_delta = set(knns.keys())

# Number of unique image ids
v_nums_intersection = v_nums_charlie.intersection(v_nums_delta)
print(len(v_nums_intersection))

# As expected, all images in the filtered valset are accounted for in the KNNs data
print(len(v_nums_charlie - v_nums_delta), len(v_nums_delta - v_nums_charlie))

39998
0 506


# Save the data

In [18]:
random.Random(123).shuffle(val_examples_filtered)

pickle.dump(val_examples_filtered, open( "valset_augmented.pickle", "wb" ) )
pickle.dump(val_examples_filtered[:10000], open( "valset_augmented_small.pickle", "wb" ) )